In [23]:
import tensorflow as tf
import keras 
import pandas as pd
from keras.applications import MobileNetV2
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Flatten
from keras.layers import AveragePooling2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Input
from keras.models import Model
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import cv2





In [ ]:
paths = '/content/face-mask-detector-master (1).zip'
imgpaths = list(paths.list_images(paths))
img = []
label = []
for imagePath in imgpaths:
	label = imagePath.split(os.path.sep)[-2]
	image = load_img(imagePath, target_size=(224, 224))
	image = img_to_array(image)
	image = preprocess_input(image)
	img.append(image)
	label.append(label)
img = np.array(img, dtype="float32")
label = np.array(label)

In [ ]:
img.shape

In [ ]:
label.shape

In [ ]:
label.unique()

In [ ]:
encoder = LabelBinarizer()
labels = encoder.fit_transform(label)
labels = to_categorical(label)

In [ ]:
train_x,test_x,train_y,test_y = train_test_split(img, label,
	test_size=0.20)

In [12]:
dgen = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [ ]:
bModel = MobileNetV2(weights="imagenet",
	input_tensor=Input(shape=(224, 224, 3)))
a = bModel.output
a = AveragePooling2D(pool_size=(7, 7))(a)
a = Flatten()(a)
a = Dense(128, activation="relu")(a)
a = Dropout(0.5)(a)
a = Dense(2, activation="softmax")(a)
model = Model(inputs=bModel.input, outputs=a)

In [14]:
for layer in bModel.layers:
	layer.trainable = False

In [ ]:
tr = model.fit(dgen.flow(train_x, train_y, batch_size=120), 
            steps_per_epoch=len(train_x) // batch_size,
			validation_data=(test_x, test_y),
			validation_steps=len(test_x) // batch_size,
			epochs=18)

In [ ]:
pred_y = model.predict(X_test, batch_size=120)
pred_y = np.argmax(pred_y, axis=1)

In [ ]:
classification_report(y_test.argmax(axis=1), y_pred, target_names=encoder.classes_)

In [ ]:
model.save("Model", save_format="h5")

In [ ]:
plt.plot(np.arange(0, epochs), tr.history["loss"], label="train_loss")
plt.plot(np.arange(0, epochs), tr.history["val_loss"], label="val_loss")
plt.title("Training and Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(loc="upper right")

In [ ]:
prototxtPath = '/content/drive/My Drive/maskclassifier/face_detector/deploy.prototxt'
weightsPath = '/content/drive/My Drive/maskclassifier/face_detector/res10_300x300_ssd_iter_140000.caffemodel'
face_model = cv2.dnn.readNet(prototxtPath, weightsPath)
model = load_model("model")

In [ ]:
image = cv2.imread('/content/drive/My Drive/maskclassifier/test/people1.png')
height, width = image.shape[:2]
blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), (104.0, 177.0, 123.0))
face_model.setInput(blob)
detections = face_model.forward()			

In [ ]:
from google.colab.patches import cv2_imshow
threshold = 0.2
person_with_mask = 0;
person_without_mask = 0;
for i in range(0, detections.shape[2]):
	score = detections[0, 0, i, 2]
	if score > threshold:
		#coordinates of the bounding box
		box = detections[0, 0, i, 3:7] * np.array([width, height, width, height])
		X_start, Y_start, X_end, Y_end = box.astype("int")
		X_start, Y_start = (max(0, X_start), max(0, Y_start))
		X_end, Y_end = (min(width - 1, X_end), min(height - 1, Y_end))

		face = image[Y_start:Y_end, X_start:X_end]
		face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)		#Convert to rgb
		face = cv2.resize(face, (224, 224))					#resize
		face = img_to_array(face)							
		face = preprocess_input(face)
		face = np.expand_dims(face, axis=0)	
	
		mask, withoutMask = model.predict(face)[0]			#To predict mask or not on the face

		if mask > withoutMask:								#determining the label
			label = "Mask"
			person_with_mask += 1
		else: 
			label = "No Mask"
			person_without_mask += 1
			
		if label == "Mask":									#determine the color
			color = (0, 255, 0)
		else:
			color = (0, 0, 255)
		label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)		#label and probability
		cv2.putText(image, label, (X_start, Y_start - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		cv2.rectangle(image, (X_start, Y_start), (X_end, Y_end), color, 2)
  
print("Number of person with mask : {}".format(person_with_mask))
print("Number of person without mask : {}".format(person_without_mask))
cv2_imshow(image)